In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
#import ESMF
import numpy as np
import pandas as pd
import xarray as xr
import pyroms

In [3]:
grd = pyroms.grid.get_ROMS_grid('SEAKp_1km')

Assuming spherical is integer 1 <class 'numpy.ma.core.MaskedArray'>
Load geographical grid from file


In [4]:
grd

In [5]:
lon2 = xr.DataArray(grd.hgrid.lon_psi)

In [6]:
ds_out = xr.Dataset({'lon': xr.DataArray(grd.hgrid.lon_psi), 'lat':xr.DataArray(grd.hgrid.lat_psi)})

In [7]:
ds_out

<xarray.Dataset>
Dimensions:  (dim_0: 281, dim_1: 325)
Dimensions without coordinates: dim_0, dim_1
Data variables:
    lon      (dim_0, dim_1) float64 212.0 212.1 212.1 ... 215.1 215.2 215.2
    lat      (dim_0, dim_1) float64 48.03 48.06 48.08 48.11 ... 65.17 65.2 65.22

In [8]:
river_data = xr.open_dataset('/import/AKWATERS/kshedstrom/hydroflow//new_2019/goa_dischargex_09012014_08312015.nc')

In [9]:
river_data

<xarray.Dataset>
Dimensions:  (time: 365, timeSeries: 14052)
Dimensions without coordinates: time, timeSeries
Data variables:
    q        (time, timeSeries) float32 ...
    lon      (timeSeries) float32 ...
    lat      (timeSeries) float32 ...
    year     (time) uint32 ...
    month    (time) uint32 ...
    day      (time) uint32 ...
Attributes:
    title:          Coastal Freshwater Discharge into the Gulf of Alaska
    summary:        Coastal FWD was modeled using a suite of physically based...
    keywords:       DISCHARGE/FLOW, ALASKA, GULF OF ALASKA
    date created:   Dataset created June 2019
    creator_name:   David Hill, Professor
    creator_email:  dfh@oregonstate.edu
    institution:    Oregon State University, Department of Civil Engineering

In [10]:
len(river_data.year)

365

In [11]:
river_data.lon[0:10]+360, river_data.lat[0:10]

(<xarray.DataArray 'lon' (timeSeries: 10)>
 array([205.27757, 205.26134, 205.29362, 205.26097, 205.3097 , 205.24455,
        205.32576, 205.22832, 205.24437, 205.34167], dtype=float32)
 Dimensions without coordinates: timeSeries,
 <xarray.DataArray 'lat' (timeSeries: 10)>
 array([56.418037, 56.41794 , 56.4271  , 56.43587 , 56.436157, 56.444733,
        56.445217, 56.44463 , 56.453697, 56.463234], dtype=float32)
 Dimensions without coordinates: timeSeries
 Attributes:
     long_name:  latitude
     units:      degrees_north)

In [12]:
len(river_data.timeSeries)

14052

In [13]:
lon2d = grd.hgrid.lon_psi
lat2d = grd.hgrid.lat_psi
Mm, Ll = lat2d.shape

In [14]:
ii, jj = np.meshgrid(range(Ll), range(Mm))
grd2 = list( zip(np.ravel(lon2d), np.ravel(lat2d)) )
idx = list( zip(np.ravel(ii), np.ravel(jj)) )

In [15]:
pts = list( zip(np.ravel(river_data.lon + 360.), np.ravel(river_data.lat)) )

In [16]:
from scipy import spatial
distance, index = spatial.KDTree(grd2).query(pts)

In [17]:
pts_coord = np.asarray(grd2)[index]
pts_idx = np.asarray(idx)[index]
pts_coord

array([[205.26645132,  56.41341473],
       [205.26645132,  56.41341473],
       [205.31316416,  56.44461114],
       ...,
       [210.62113403,  61.55675419],
       [210.68197962,  61.52861032],
       [210.68197962,  61.52861032]])

In [18]:
pts_idx

array([[ 84, 216],
       [ 84, 216],
       [ 85, 216],
       ...,
       [224, 250],
       [224, 249],
       [224, 249]])

In [19]:
indices = xr.Dataset({'i': (['river'], pts_idx[:,0]), 'j': (['river'], pts_idx[:,1])})

In [20]:
indices

<xarray.Dataset>
Dimensions:  (river: 14052)
Dimensions without coordinates: river
Data variables:
    i        (river) int64 84 84 85 84 85 85 85 ... 223 223 224 224 224 224 224
    j        (river) int64 216 216 216 216 216 217 ... 249 250 249 250 249 249

In [21]:
#indices.to_netcdf('indices.nc')

In [22]:
import pyroms_toolbox
width = 1
idx = []
idy = []
maskl = grd.hgrid.mask_rho.copy()
for w in range(width):
    lit = pyroms_toolbox.get_littoral2(maskl)
    idx.extend(lit[0])
    idy.extend(lit[1])
    maskl[lit] = 0

In [23]:
littoral_idx = (np.array(idx), np.array(idy))
maskl = np.zeros(grd.hgrid.mask_rho.shape)
maskl[littoral_idx] = 1

In [24]:
mask_idx = np.where(grd.hgrid.mask_rho == 0)

Make the runoff file

In [25]:
import netCDF4 as netCDF
from datetime import datetime
nt = len(river_data.time)
Mp, Lp = grd.hgrid.mask_rho.shape
#spval = -1e30
spval = -9999.
runoff_raw = np.zeros((Mp,Lp))
runoff = np.zeros((Mp,Lp))

# create runoff file
runoff_file = 'runoff_NGOA_Hill_2015.nc'
nc = netCDF.Dataset(runoff_file, 'w', format='NETCDF3_64BIT')
nc.Description = 'Hill & Beamer monthly climatology river discharge'
nc.Author = 'make_runoff_clim.py'
nc.Created = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
nc.title = 'Hill & Beamer river discharge'

# create dimensions and variables
nc.createDimension('xi_rho', np.size(grd.hgrid.mask_rho,1))
nc.createDimension('eta_rho', np.size(grd.hgrid.mask_rho,0))
nc.createDimension('runoff_time', None)

nc.createVariable('lon_rho', 'f8', ('eta_rho', 'xi_rho'))
nc.variables['lon_rho'].long_name = 'longitude of RHO-points'
nc.variables['lon_rho'].units = 'degree_east'
nc.variables['lon_rho'].field = 'lon_rho, scalar'
nc.variables['lon_rho'][:] = grd.hgrid.lon_rho

nc.createVariable('lat_rho', 'f8', ('eta_rho', 'xi_rho'))
nc.variables['lat_rho'].long_name = 'latitude of RHO-points'
nc.variables['lat_rho'].units = 'degree_north'
nc.variables['lat_rho'].field = 'lat_rho, scalar'
nc.variables['lat_rho'][:] = grd.hgrid.lat_rho

nc.createVariable('runoff_time', 'f8', ('runoff_time'))
nc.variables['runoff_time'].long_name = 'time'
nc.variables['runoff_time'].units = 'days since 1900-01-01 00:00:00'
#nc.variables['runoff_time'].cycle_length = 365.25

nc.createVariable('Runoff_raw', 'f8', ('runoff_time', 'eta_rho', 'xi_rho'), fill_value=spval)
nc.variables['Runoff_raw'].long_name = 'Dai_Trenberth River Runoff raw'
nc.variables['Runoff_raw'].units = 'kg/s/m^2'

nc.createVariable('Runoff', 'f8', ('runoff_time', 'eta_rho', 'xi_rho'), fill_value=spval)
nc.variables['Runoff'].long_name = 'Dai_Trenberth River Runoff'
nc.variables['Runoff'].units = 'kg/s/m^2'


In [26]:
import datetime
date1 = datetime.date(1900, 1, 1)

In [27]:
river_data.q.shape

(365, 14052)

In [28]:
nt

365

In [29]:
river_data.q

<xarray.DataArray 'q' (time: 365, timeSeries: 14052)>
[5128980 values with dtype=float32]
Dimensions without coordinates: time, timeSeries
Attributes:
    long_name:       Daily discharge
    units:           cubic meters per day
    coordinates:     time lat lon
    standard_name:   runoff_flux
    standard_units:  kg m-2 s-1

In [30]:
for t in range(nt):
    flow = np.sum(river_data.q[t,:])
    dater = datetime.date(river_data.year[t], river_data.month[t], river_data.day[t])
    time = (dater - date1).days
    print('Remapping runoff for time %f' %time)
    runoff_raw = np.zeros((Mp,Lp))
    runoff = np.zeros((Mp,Lp))
    for ri in range(len(river_data.timeSeries)):
        i = indices.i[ri]
        j = indices.j[ri]
        if (i > 323 or j == 0):
   #         print("overflow", i, j, ri)
            continue
        runoff_raw[j,i] += float(river_data.q[t,ri])
        
    nflow = np.sum(runoff_raw)
    print('time', t, flow/nflow)
    idx = np.where(runoff_raw != 0)
    runoff = pyroms_toolbox.move_runoff(runoff_raw, \
                  np.array(idx).T + 1, np.array(littoral_idx).T + 1, maskl, \
                  grd.hgrid.x_rho, grd.hgrid.y_rho, grd.hgrid.dx, grd.hgrid.dy)

# write data in destination file
    nc.variables['Runoff'][t] = runoff
    nc.variables['Runoff_raw'][t] = runoff_raw
    nc.variables['runoff_time'][t] = time

    if t==5:
        print('Sum 3', np.sum(runoff_raw))
        print('Sum 4', np.sum(runoff))

nc.close()

Remapping runoff for time 41881.000000
time 0 <xarray.DataArray 'q' ()>
array(1.09366047)
Remapping runoff for time 41882.000000
time 1 <xarray.DataArray 'q' ()>
array(1.10249175)
Remapping runoff for time 41883.000000
time 2 <xarray.DataArray 'q' ()>
array(1.10282355)
Remapping runoff for time 41884.000000
time 3 <xarray.DataArray 'q' ()>
array(1.09769306)
Remapping runoff for time 41885.000000
time 4 <xarray.DataArray 'q' ()>
array(1.08290235)
Remapping runoff for time 41886.000000
time 5 <xarray.DataArray 'q' ()>
array(1.06923241)
Sum 3 3961187567.407959
Sum 4 3961078529.1998944
Remapping runoff for time 41887.000000
time 6 <xarray.DataArray 'q' ()>
array(1.06222854)
Remapping runoff for time 41888.000000
time 7 <xarray.DataArray 'q' ()>
array(1.05991245)
Remapping runoff for time 41889.000000
time 8 <xarray.DataArray 'q' ()>
array(1.05873317)
Remapping runoff for time 41890.000000
time 9 <xarray.DataArray 'q' ()>
array(1.05626737)
Remapping runoff for time 41891.000000
time 10 <xar

time 90 <xarray.DataArray 'q' ()>
array(1.16780907)
Remapping runoff for time 41972.000000
time 91 <xarray.DataArray 'q' ()>
array(1.17238882)
Remapping runoff for time 41973.000000
time 92 <xarray.DataArray 'q' ()>
array(1.15735222)
Remapping runoff for time 41974.000000
time 93 <xarray.DataArray 'q' ()>
array(1.1326182)
Remapping runoff for time 41975.000000
time 94 <xarray.DataArray 'q' ()>
array(1.11692977)
Remapping runoff for time 41976.000000
time 95 <xarray.DataArray 'q' ()>
array(1.10064032)
Remapping runoff for time 41977.000000
time 96 <xarray.DataArray 'q' ()>
array(1.10175838)
Remapping runoff for time 41978.000000
time 97 <xarray.DataArray 'q' ()>
array(1.10860169)
Remapping runoff for time 41979.000000
time 98 <xarray.DataArray 'q' ()>
array(1.10363754)
Remapping runoff for time 41980.000000
time 99 <xarray.DataArray 'q' ()>
array(1.13665236)
Remapping runoff for time 41981.000000
time 100 <xarray.DataArray 'q' ()>
array(1.17686993)
Remapping runoff for time 41982.000000

time 180 <xarray.DataArray 'q' ()>
array(1.14154275)
Remapping runoff for time 42062.000000
time 181 <xarray.DataArray 'q' ()>
array(1.16113499)
Remapping runoff for time 42063.000000
time 182 <xarray.DataArray 'q' ()>
array(1.17454497)
Remapping runoff for time 42064.000000
time 183 <xarray.DataArray 'q' ()>
array(1.16845435)
Remapping runoff for time 42065.000000
time 184 <xarray.DataArray 'q' ()>
array(1.15633549)
Remapping runoff for time 42066.000000
time 185 <xarray.DataArray 'q' ()>
array(1.19037588)
Remapping runoff for time 42067.000000
time 186 <xarray.DataArray 'q' ()>
array(1.2456435)
Remapping runoff for time 42068.000000
time 187 <xarray.DataArray 'q' ()>
array(1.23013695)
Remapping runoff for time 42069.000000
time 188 <xarray.DataArray 'q' ()>
array(1.20421077)
Remapping runoff for time 42070.000000
time 189 <xarray.DataArray 'q' ()>
array(1.20155296)
Remapping runoff for time 42071.000000
time 190 <xarray.DataArray 'q' ()>
array(1.22310828)
Remapping runoff for time 42

time 270 <xarray.DataArray 'q' ()>
array(1.17155959)
Remapping runoff for time 42152.000000
time 271 <xarray.DataArray 'q' ()>
array(1.16643678)
Remapping runoff for time 42153.000000
time 272 <xarray.DataArray 'q' ()>
array(1.16012406)
Remapping runoff for time 42154.000000
time 273 <xarray.DataArray 'q' ()>
array(1.15358882)
Remapping runoff for time 42155.000000
time 274 <xarray.DataArray 'q' ()>
array(1.14731168)
Remapping runoff for time 42156.000000
time 275 <xarray.DataArray 'q' ()>
array(1.14051519)
Remapping runoff for time 42157.000000
time 276 <xarray.DataArray 'q' ()>
array(1.13178619)
Remapping runoff for time 42158.000000
time 277 <xarray.DataArray 'q' ()>
array(1.12670036)
Remapping runoff for time 42159.000000
time 278 <xarray.DataArray 'q' ()>
array(1.12562663)
Remapping runoff for time 42160.000000
time 279 <xarray.DataArray 'q' ()>
array(1.1321481)
Remapping runoff for time 42161.000000
time 280 <xarray.DataArray 'q' ()>
array(1.14023657)
Remapping runoff for time 42

time 360 <xarray.DataArray 'q' ()>
array(1.06971399)
Remapping runoff for time 42242.000000
time 361 <xarray.DataArray 'q' ()>
array(1.07833663)
Remapping runoff for time 42243.000000
time 362 <xarray.DataArray 'q' ()>
array(1.10463022)
Remapping runoff for time 42244.000000
time 363 <xarray.DataArray 'q' ()>
array(1.12968823)
Remapping runoff for time 42245.000000
time 364 <xarray.DataArray 'q' ()>
array(1.15139259)
